<a href="https://colab.research.google.com/github/SAHIL9581/LIVE-AI-CLASSES/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip uninstall torchtext torch -y
!pip cache purge  # Clear cached packages to avoid conflicts

# Install PyTorch for CUDA 11.8 (modify if using a different CUDA version)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install the compatible torchtext version
!pip install torchtext==0.16.0  # Make sure it matches the PyTorch version

import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from torch.utils.data import Dataset, DataLoader
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab

# Download stopwords if not already present
nltk.download("stopwords")
nltk.download("punkt_tab")



Found existing installation: torch 2.5.1+cu118
Uninstalling torch-2.5.1+cu118:
  Successfully uninstalled torch-2.5.1+cu118
Files removed: 26
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 838.4/838.4 MB 2.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.5.1+cu118 which is incompatible.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 8.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.1.0
    Uninstalling triton-3.1.0:
      Successfully uninstalled triton-3.1.0
  Attempting uninstall: torch
    Found existing installation: torch 2.5.1+cu118
    Uninstalling torch-2.5.1+cu118:
      Successfully uninstalled torch-2.5.1+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1+cu124 requires torch==2.5.1, but you have torch 2.1.0 which is incompatible.
torchvision 0.20.1+cu124 requires torch==2.5.1, but you have torch 2.1.0 which is incompatible.


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [2]:
import zipfile
import os

# Replace 'your_file.zip' with the name of your uploaded zip file
zip_file = "/content/blogs.zip"

# Create a directory for extraction
extract_dir = "/content/extracted_folder"

# Extract the zip file
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to {extract_dir}")


Files extracted to /content/extracted_folder


In [3]:
import os
import xml.etree.ElementTree as ET
import pandas as pd

# Folder containing XML files
folder_path = "/content/extracted_folder/blogs"  # Change to your actual folder path

# List to store extracted data
all_data = []
error_files = []

# Loop through all XML files
for filename in os.listdir(folder_path):
    if filename.endswith(".xml"):  # Process only XML files
        file_path = os.path.join(folder_path, filename)

        try:
            # Extract metadata from filename
            parts = filename.split('.')
            if len(parts) < 3:
                continue  # Skip invalid files

            author_id, gender, age = parts[0], parts[1], int(parts[2])

            # Read XML file with error handling
            with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                xml_content = f.read()

            # Parse XML
            tree = ET.ElementTree(ET.fromstring(xml_content))
            root = tree.getroot()

            # Extract blog posts
            for post in root.findall("post"):
                text = post.text.strip() if post.text else ""  # Extract text
                all_data.append([author_id, gender, age, text])

        except Exception as e:
            error_files.append((filename, str(e)))  # Store error details

# Convert to DataFrame
df = pd.DataFrame(all_data, columns=["Author_ID", "Gender", "Age", "Text"])

# Save extracted data
csv_path = "blog_data.csv"
df.to_csv(csv_path, index=False)

print(f"Data extraction complete. Saved to {csv_path}.")
print(f"Skipped {len(error_files)} files due to errors.")

# Log files that caused errors
if error_files:
    error_log_path = "error_files.log"
    with open(error_log_path, "w") as f:
        for file, error in error_files:
            f.write(f"{file}: {error}\n")
    print(f"Error log saved to {error_log_path}.")

Data extraction complete. Saved to blog_data.csv.
Skipped 11449 files due to errors.
Error log saved to error_files.log.


In [7]:
# ==================== STEP 1: LOAD & PREPROCESS DATA ====================

# Load dataset
csv_path = "blog_data.csv"
df = pd.read_csv(csv_path)

# Text cleaning function
def clean_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\W+', ' ', text)  # Remove special characters
    text = re.sub(r'\d+', '', text)  # Remove numbers
    tokens = word_tokenize(text)  # Tokenize
    tokens = [word for word in tokens if word not in stopwords.words('english')]  # Remove stopwords
    return " ".join(tokens)

df["Cleaned_Text"] = df["Text"].astype(str).apply(clean_text)

# Save cleaned data
cleaned_csv_path = "cleaned_blog_data.csv"
df.to_csv(cleaned_csv_path, index=False)



In [8]:
# ==================== STEP 2: TOKENIZATION & EMBEDDINGS ====================

# Tokenizer
tokenizer = get_tokenizer("basic_english")
df["Tokenized_Text"] = df["Cleaned_Text"].astype(str).apply(tokenizer)

# Build vocabulary
counter = Counter()
for tokens in df["Tokenized_Text"]:
    counter.update(tokens)

# Create vocabulary (min frequency = 2)
vocab_obj = vocab(counter, min_freq=2, specials=["<unk>", "<pad>"])
vocab_obj.set_default_index(vocab_obj["<unk>"])

# Convert text to numerical format
df["Numerical_Text"] = df["Tokenized_Text"].apply(lambda x: [vocab_obj[word] for word in x])

# Padding function
MAX_LEN = 100  # Adjust based on dataset
def pad_sequence(seq, max_len=MAX_LEN):
    if len(seq) < max_len:
        seq += [vocab_obj["<pad>"]] * (max_len - len(seq))  # Pad
    return seq[:max_len]  # Truncate

df["Padded_Text"] = df["Numerical_Text"].apply(lambda x: pad_sequence(x))

# Save processed data
processed_csv_path = "tokenized_blog_data.csv"
df.to_csv(processed_csv_path, index=False)


In [12]:
# ==================== STEP 3: CREATE DATASET & DATALOADER ====================

class BlogDataset(Dataset):
    def __init__(self, dataframe, indices): # Add indices argument
        self.data = dataframe
        self.indices = indices # Store indices
        self.gender_map = {"male": 0, "female": 1}  # Binary classification
        self.age_map = {10: 0, 20: 1, 30: 2, 40: 3, 50: 4}  # Age groups
        self.topic_map = {i: i for i in range(10)}  # Example: 10 topics

    def __len__(self):
        return len(self.indices) # Use length of indices

    def __getitem__(self, idx):
        # Use indices to access data from the original dataframe
        data_idx = self.indices[idx]
        text = torch.tensor(self.data.iloc[data_idx]["Padded_Text"], dtype=torch.long)
        gender = torch.tensor(self.gender_map[self.data.iloc[data_idx]["Gender"]], dtype=torch.long)
        age = torch.tensor(self.age_map[min(self.data.iloc[data_idx]["Age"] // 10 * 10, 50)], dtype=torch.long)  # Bucket ages
        topic = torch.tensor(self.topic_map[self.data.iloc[data_idx]["Author_ID"] % 10], dtype=torch.long)  # Example topic
        return text, gender, age, topic

# Split dataset
train_size = int(0.8 * len(df))
val_size = len(df) - train_size
train_data, val_data = torch.utils.data.random_split(df, [train_size, val_size])

# Create DataLoaders
# Pass indices to BlogDataset
train_dataset = BlogDataset(df, train_data.indices) # Pass original dataframe and indices
val_dataset = BlogDataset(df, val_data.indices)  # Pass original dataframe and indices

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)



In [13]:
# ==================== STEP 4: BUILD MULTI-TASK CNN MODEL ====================

class MultiTaskCNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_filters, filter_sizes, hidden_dim, output_dims, dropout=0.5):
        super(MultiTaskCNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.convs = nn.ModuleList([nn.Conv1d(embed_dim, num_filters, k) for k in filter_sizes])
        self.fc_gender = nn.Linear(num_filters * len(filter_sizes), output_dims["gender"])
        self.fc_age = nn.Linear(num_filters * len(filter_sizes), output_dims["age"])
        self.fc_topic = nn.Linear(num_filters * len(filter_sizes), output_dims["topic"])
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        x = x.permute(0, 2, 1)
        conv_results = [F.relu(conv(x)).max(dim=2)[0] for conv in self.convs]
        x = torch.cat(conv_results, dim=1)
        x = self.dropout(x)
        return self.fc_gender(x), self.fc_age(x), self.fc_topic(x)

# Initialize Model
vocab_size = len(vocab_obj)
embed_dim = 100
num_filters = 128
filter_sizes = [3, 4, 5]
hidden_dim = 256
output_dims = {"gender": 2, "age": 5, "topic": 10}
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = MultiTaskCNN(vocab_size, embed_dim, num_filters, filter_sizes, hidden_dim, output_dims).to(device)
print(model)


MultiTaskCNN(
  (embedding): Embedding(94983, 100)
  (convs): ModuleList(
    (0): Conv1d(100, 128, kernel_size=(3,), stride=(1,))
    (1): Conv1d(100, 128, kernel_size=(4,), stride=(1,))
    (2): Conv1d(100, 128, kernel_size=(5,), stride=(1,))
  )
  (fc_gender): Linear(in_features=384, out_features=2, bias=True)
  (fc_age): Linear(in_features=384, out_features=5, bias=True)
  (fc_topic): Linear(in_features=384, out_features=10, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)


In [14]:
# ==================== STEP 5: TRAINING & EVALUATION ====================

# Loss functions & Optimizer
criterion_gender = nn.CrossEntropyLoss()
criterion_age = nn.CrossEntropyLoss()
criterion_topic = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss, total_correct_gender, total_correct_age, total_correct_topic = 0, 0, 0, 0
    for texts, gender_labels, age_labels, topic_labels in train_loader:
        texts, gender_labels, age_labels, topic_labels = texts.to(device), gender_labels.to(device), age_labels.to(device), topic_labels.to(device)

        optimizer.zero_grad()
        gender_out, age_out, topic_out = model(texts)

        loss_gender = criterion_gender(gender_out, gender_labels)
        loss_age = criterion_age(age_out, age_labels)
        loss_topic = criterion_topic(topic_out, topic_labels)
        loss = loss_gender + loss_age + loss_topic

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        total_correct_gender += (gender_out.argmax(1) == gender_labels).sum().item()
        total_correct_age += (age_out.argmax(1) == age_labels).sum().item()
        total_correct_topic += (topic_out.argmax(1) == topic_labels).sum().item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(train_loader):.4f}, Gender Acc: {total_correct_gender/len(train_dataset):.4f}, Age Acc: {total_correct_age/len(train_dataset):.4f}, Topic Acc: {total_correct_topic/len(train_dataset):.4f}")

# Save the trained model
torch.save(model.state_dict(), "multi_task_cnn.pth")
print("Model training complete. Model saved!")



Epoch 1/5, Loss: 4.1409, Gender Acc: 0.5604, Age Acc: 0.5409, Topic Acc: 0.1083
Epoch 2/5, Loss: 3.9433, Gender Acc: 0.5949, Age Acc: 0.6118, Topic Acc: 0.1183
Epoch 3/5, Loss: 3.8128, Gender Acc: 0.6389, Age Acc: 0.6540, Topic Acc: 0.1260
Epoch 4/5, Loss: 3.6666, Gender Acc: 0.6872, Age Acc: 0.6897, Topic Acc: 0.1435
Epoch 5/5, Loss: 3.5217, Gender Acc: 0.7236, Age Acc: 0.7188, Topic Acc: 0.1631
Model training complete. Model saved!


In [15]:
# Load the trained model
model.load_state_dict(torch.load("multi_task_cnn.pth"))
model.to(device)
model.eval()  # Set to evaluation mode

# Evaluation function
def evaluate_model(model, dataloader):
    total_loss, total_correct_gender, total_correct_age, total_correct_topic = 0, 0, 0, 0
    criterion_gender = nn.CrossEntropyLoss()
    criterion_age = nn.CrossEntropyLoss()
    criterion_topic = nn.CrossEntropyLoss()

    with torch.no_grad():
        for texts, gender_labels, age_labels, topic_labels in dataloader:
            texts, gender_labels, age_labels, topic_labels = texts.to(device), gender_labels.to(device), age_labels.to(device), topic_labels.to(device)

            gender_out, age_out, topic_out = model(texts)

            loss_gender = criterion_gender(gender_out, gender_labels)
            loss_age = criterion_age(age_out, age_labels)
            loss_topic = criterion_topic(topic_out, topic_labels)
            loss = loss_gender + loss_age + loss_topic

            total_loss += loss.item()
            total_correct_gender += (gender_out.argmax(1) == gender_labels).sum().item()
            total_correct_age += (age_out.argmax(1) == age_labels).sum().item()
            total_correct_topic += (topic_out.argmax(1) == topic_labels).sum().item()

    print(f"Validation Loss: {total_loss/len(dataloader):.4f}")
    print(f"Gender Accuracy: {total_correct_gender/len(val_dataset):.4f}")
    print(f"Age Accuracy: {total_correct_age/len(val_dataset):.4f}")
    print(f"Topic Accuracy: {total_correct_topic/len(val_dataset):.4f}")

# Run evaluation
evaluate_model(model, val_loader)



Validation Loss: 3.7642
Gender Accuracy: 0.6758
Age Accuracy: 0.6584
Topic Accuracy: 0.1524


In [16]:
# Prediction function
def predict_blog_post(model, text):
    model.eval()
    text = clean_text(text)  # Preprocess
    tokens = tokenizer(text)  # Tokenize
    numerical_text = [vocab_obj[word] for word in tokens]  # Convert to numerical
    padded_text = pad_sequence(numerical_text)  # Pad sequence
    input_tensor = torch.tensor(padded_text, dtype=torch.long).unsqueeze(0).to(device)  # Convert to tensor

    with torch.no_grad():
        gender_out, age_out, topic_out = model(input_tensor)

    predicted_gender = "Male" if torch.argmax(gender_out) == 0 else "Female"
    predicted_age = (torch.argmax(age_out).item() * 10, (torch.argmax(age_out).item() + 1) * 10)  # Age range
    predicted_topic = torch.argmax(topic_out).item()

    return predicted_gender, predicted_age, predicted_topic

# Example Prediction
sample_text = "I love playing video games and watching sci-fi movies!"
pred_gender, pred_age, pred_topic = predict_blog_post(model, sample_text)

print(f"Predicted Gender: {pred_gender}")
print(f"Predicted Age Group: {pred_age}")
print(f"Predicted Topic ID: {pred_topic}")



Predicted Gender: Male
Predicted Age Group: (10, 20)
Predicted Topic ID: 6
